Based on the following video: [How-to Use The Reddit API in Python by James Briggs](https://www.youtube.com/watch?v=FdjVoOf9HN4)
1. Create a reddit application [here](https://www.reddit.com/prefs/apps) to get a client_id and client_secret


In [ ]:
import requests
import dotenv
import os

In [ ]:
## Authenticate with the reddit API

dotenv.load_dotenv()
REDDIT_CLIENT_ID = os.getenv("REDDIT_CLIENT_ID")  # Fixed: was getting CLIENT_SECRET
REDDIT_CLIENT_SECRET = os.getenv("REDDIT_CLIENT_SECRET")
REDDIT_USERNAME = os.getenv("REDDIT_USERNAME")
REDDIT_PASSWORD = os.getenv("REDDIT_PASSWORD")

res = requests.post('https://www.reddit.com/api/v1/access_token',
    auth=requests.auth.HTTPBasicAuth(REDDIT_CLIENT_ID, REDDIT_CLIENT_SECRET),
    data={
        'grant_type': 'password',
        'username': REDDIT_USERNAME,
        'password': REDDIT_PASSWORD
    },
    headers={'User-Agent': 'MyAPI/0.0.1'}
)

TOKEN = res.json()['access_token']

headers = {
    **{'User-Agent': 'MyAPI/0.0.1'}, 
    **{'Authorization': f'bearer {TOKEN}'}
}

In [ ]:
# Access user information

res = requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)

print(f'username: {res.json()['subreddit']['display_name']}')

Username: u_Verusauxilium
